In [188]:
%load_ext autoreload
%autoreload 2
from backtest.domains.strategy_result import StrategyResult,StrategyResultColumnType
from backtest.domains.stockdata import StockData
import pandas as pd
import numpy as np
# stockdata 불러오기 

def strategy_result_data(name='strategy1'):
    sample_dict = {'date': ['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05'],
                   name: [(StrategyResultColumnType.KEEP, 100),
                                 (StrategyResultColumnType.BUY, 100),
                                 (StrategyResultColumnType.SELL, 100),
                                 (StrategyResultColumnType.SELL, 100),
                                 (StrategyResultColumnType.BUY, 100),]}
    return StrategyResult.from_dict(sample_dict, target='ALL')

def dict_stock_data_list():
    return [{'open': [0.00000, '1.11111', '3.3333', '0.0000', '1.0000'],
            'high': [0.00000, '1.11111', '3.3333', '0.0000', '1.0000'],
             'low': [0.00000, '1.11111', '3.3333', '0.0000', '1.0000'],
             'close': [0.00000, '1.11111', '3.3333', '0.0000', '1.0000'],
             'volume': [0.00000, '1.11111', '3.3333', '0.0000', '1.0000'],
             'date': ['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05']},
            {'open': [0.00000, '1.11111', '2.2222', '0.0000', '1.0000'],
            'high': [0.00000, '1.11111', '2.2222', '0.0000', '1.0000'],
             'low': [0.00000, '1.11111', '2.2222', '0.0000', '1.0000'],
             'close': [0.00000, '1.11111', '2.2222', '0.0000', '1.0000'],
             'volume': [0.00000, '1.11111', '2.2222', '0.0000', '1.0000'],
             'date': ['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05']},
            {'open': [0.00000, '1.11111', '3.3333', '0.0000', '1.0000'],
            'high': [0.00000, '1.11111', '4.4444', '0.0000', '1.0000'],
             'low': [0.00000, '1.11111', '4.4444', '0.0000', '1.0000'],
             'close': [0.00000, '1.11111', '4.4444', '0.0000', '1.0000'],
             'volume': [0.00000, '1.11111', '5.5555', '0.0000', '1.0000'],
             'date': ['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05']}]


def stockdata_list():
    dict_stock=dict_stock_data_list()
    stockdata1 = StockData.from_dict(
        dict_data=dict_stock[0], symbol='teststock1')
    stockdata2 = StockData.from_dict(
        dict_data=dict_stock[1], symbol='teststock2')
    stockdata3 = StockData.from_dict(
        dict_data=dict_stock[2], symbol='teststock3')
    return [
        stockdata1, stockdata2, stockdata3
    ]


stock_data1=stockdata_list()[0]
stock_data1_raw=stock_data1.data
strategy_results = []
for i in range(3):
    strategy_results.append(strategy_result_data('strategy{}'.format(i)))
strategy_total_result = pd.DataFrame(index=pd.DatetimeIndex([]))
for strategy_result in strategy_results:
    strategy_total_result = strategy_total_result.join(strategy_result.value, how='outer')

stock_data1_raw['total'] = strategy_total_result.apply(
    lambda row: sum_strategy(row), axis=1)

stock_data1_raw['bucket']=np.nan
stock_data1_raw['bucket'].astype('object')
stock_data1_raw.at[stock_data1_raw.index[0], 'bucket'] = 'DUMMY'

for index in stock_data1_raw.index:
    stock_data1_raw.at[index,'bucket'] = []
    print(stock_data1_raw.at[index, 'bucket'])



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{<StrategyResultColumnType.KEEP: 3>: 300, <StrategyResultColumnType.SELL: 2>: 0, <StrategyResultColumnType.BUY: 1>: 0}
{<StrategyResultColumnType.KEEP: 3>: 0, <StrategyResultColumnType.SELL: 2>: 0, <StrategyResultColumnType.BUY: 1>: 300}
{<StrategyResultColumnType.KEEP: 3>: 0, <StrategyResultColumnType.SELL: 2>: 300, <StrategyResultColumnType.BUY: 1>: 0}
{<StrategyResultColumnType.KEEP: 3>: 0, <StrategyResultColumnType.SELL: 2>: 300, <StrategyResultColumnType.BUY: 1>: 0}
{<StrategyResultColumnType.KEEP: 3>: 0, <StrategyResultColumnType.SELL: 2>: 0, <StrategyResultColumnType.BUY: 1>: 300}
[]
[]
[]
[]
[]


In [91]:
def sum_strategy(series : pd.Series):
    total_result = {StrategyResultColumnType.KEEP:0,
                    StrategyResultColumnType.SELL:0,
                    StrategyResultColumnType.BUY:0}
    for idx in series.index:
        type,weight=series[idx]
        total_result[type]+=weight
    print(total_result)
    return max(total_result,key=total_result.get)


strate=strategy_total_result.apply(
    lambda row: sum_strategy(row), axis=1)
#            for column in strategy_total_result.columns:
#                strategy_total_result[column] = strategy_total_result[column].fillna(
#                    {i: (StrategyResultColumnType.KEEP, 0) for i in strategy_total_result.index})


{<StrategyResultColumnType.KEEP: 3>: 300, <StrategyResultColumnType.SELL: 2>: 0, <StrategyResultColumnType.BUY: 1>: 0}
{<StrategyResultColumnType.KEEP: 3>: 0, <StrategyResultColumnType.SELL: 2>: 0, <StrategyResultColumnType.BUY: 1>: 300}
{<StrategyResultColumnType.KEEP: 3>: 0, <StrategyResultColumnType.SELL: 2>: 300, <StrategyResultColumnType.BUY: 1>: 0}
{<StrategyResultColumnType.KEEP: 3>: 0, <StrategyResultColumnType.SELL: 2>: 300, <StrategyResultColumnType.BUY: 1>: 0}
{<StrategyResultColumnType.KEEP: 3>: 0, <StrategyResultColumnType.SELL: 2>: 0, <StrategyResultColumnType.BUY: 1>: 300}


StockData(symbol='teststock1', data=date
2022-01-01    StrategyResultColumnType.KEEP
2022-01-02     StrategyResultColumnType.BUY
2022-01-03    StrategyResultColumnType.SELL
2022-01-04    StrategyResultColumnType.SELL
2022-01-05     StrategyResultColumnType.BUY
dtype: object)

In [12]:

strategy_total_result
for column in strategy_total_result.columns:
    strategy_total_result[column] = strategy_total_result[column].fillna(
        {i: (StrategyResultColumnType.KEEP, 0) for i in strategy_total_result.index})
strategy_total_result


,strategy1
date,
2022-01-01,"(StrategyResultColumnType.KEEP, 100)"
2022-01-02,"(StrategyResultColumnType.BUY, 100)"
2022-01-03,"(StrategyResultColumnType.SELL, 100)"
2022-01-04,"(StrategyResultColumnType.SELL, 100)"
2022-01-05,"(StrategyResultColumnType.BUY, 100)"


In [3]:
stockdata_binance.value

StockData(symbol='', data=                         open      high       low     close         volume
date                                                                      
2017-12-02 09:00:00  10775.04  11190.00  10620.00  10890.01    4765.439757
2017-12-03 09:00:00  10902.69  11825.00  10500.00  11165.41    5346.636524
2017-12-04 09:00:00  11165.41  11600.00  10802.00  11579.00    4663.424562
2017-12-05 09:00:00  11571.03  11853.00  11447.68  11699.99    5550.732055
2017-12-06 09:00:00  11699.99  13615.23  11665.58  13550.05    6707.946319
...                       ...       ...       ...       ...            ...
2023-02-01 09:00:00  23125.13  23812.66  22760.23  23732.66  310790.422710
2023-02-02 09:00:00  23731.41  24255.00  23363.27  23488.94  364177.207510
2023-02-03 09:00:00  23489.33  23715.70  23204.62  23431.90  332571.029040
2023-02-04 09:00:00  23431.90  23587.78  23253.96  23326.84  166126.472950
2023-02-05 09:00:00  23327.66  23390.00  23230.00  23371.89   34702.823180

In [4]:
stockdata_upbit.value

StockData(symbol='', data=                           open        high         low       close  \
date                                                                  
2017-12-01 09:00:00  11935000.0  12680000.0  11201000.0  12532000.0   
2017-12-02 09:00:00  12532000.0  13180000.0  12350000.0  12971000.0   
2017-12-03 09:00:00  12970000.0  13529000.0  12840000.0  13128000.0   
2017-12-04 09:00:00  13120000.0  13688000.0  13119000.0  13684000.0   
2017-12-05 09:00:00  13683000.0  14794000.0  13612000.0  14645000.0   
...                         ...         ...         ...         ...   
2023-02-01 09:00:00  29052000.0  29740000.0  28626000.0  29620000.0   
2023-02-02 09:00:00  29599000.0  30285000.0  29530000.0  29600000.0   
2023-02-03 09:00:00  29601000.0  29734000.0  29287000.0  29432000.0   
2023-02-04 09:00:00  29432000.0  29650000.0  29349000.0  29420000.0   
2023-02-05 09:00:00  29398000.0  29528000.0  29350000.0  29450000.0   

                           volume  
date          

In [5]:
# strategy function 제작
import pandas as pd
from backtest.domains.strategy_result import StrategyResult, StrategyResultColumnType
from backtest.domains.strategy import Strategy

def strategy_function1(strategy: Strategy):
    response = StrategyResult(value=pd.DataFrame(
        index=strategy.data.index, columns=[strategy.name]))
    response.value[strategy.name] = [(
        StrategyResultColumnType.SELL, strategy.weight)] * len(strategy)
    return response

In [6]:
from backtest.use_cases.strategy_execute import strategy_execute
from backtest.use_cases.strategy_execute import _basic_function
from backtest.domains.strategy import Strategy,StrategyType

strategy1 = Strategy(name='basic_strategy',type=StrategyType.with_stockdata,data=stockdata_upbit.value,weight=100)
strategy2 = Strategy(name='basic_strategy2',type=StrategyType.with_stockdata,data=stockdata_upbit.value,weight=300)

strategy_result1 = strategy_execute(strategy1)
strategy_result2 = strategy_execute(strategy2,strategy_fucntion=strategy_function1)

In [7]:
strategy_result1.value
# StrategyResultColumnType 종류
# KEEP, SELL, BUY

StrategyResult(value=                                           basic_strategy
date                                                     
2017-12-01 09:00:00  (StrategyResultColumnType.KEEP, 100)
2017-12-02 09:00:00  (StrategyResultColumnType.KEEP, 100)
2017-12-03 09:00:00  (StrategyResultColumnType.KEEP, 100)
2017-12-04 09:00:00  (StrategyResultColumnType.KEEP, 100)
2017-12-05 09:00:00  (StrategyResultColumnType.KEEP, 100)
...                                                   ...
2023-02-01 09:00:00  (StrategyResultColumnType.KEEP, 100)
2023-02-02 09:00:00  (StrategyResultColumnType.KEEP, 100)
2023-02-03 09:00:00  (StrategyResultColumnType.KEEP, 100)
2023-02-04 09:00:00  (StrategyResultColumnType.KEEP, 100)
2023-02-05 09:00:00  (StrategyResultColumnType.KEEP, 100)

[1893 rows x 1 columns])

In [74]:
strategy_result2.value

NameError: name 'strategy_result2' is not defined

In [73]:
#https: // stackoverflow.com/questions/31458794/python-using-iterrows-to-create-columns
df = pd.DataFrame({'A': [1, 2, 3], 'B': ['a', 'b', 'c']})
total_string=''
for index,row in df.iterrows():
    total_string+=row['B']
    df.loc[index,'C'] = total_string

df


,A,B,C
0,1,a,a
1,2,b,ab
2,3,c,abc
